[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/learn/experimental/algos-and-libraries/offline-evaluation/offline-evaluation.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/learn/experimental/algos-and-libraries/offline-evaluation/offline-evaluation.ipynb)

# Evaluation Metrics for Information Retrieval

When navigating online, we often use some sort of search tool. Like when looking for a video on YouTube, buying a product on Amazon, or trying to find something with Google. The result of these searches are a list of top-$k$ relevant results, which should ideally meet the user's query.

**Evaluation measures** assess how well the search results meet the user's query. 

They can be divided into *online metrics* and *offline metrics*. 

Online metrics measure the users' interactions with the search system, while offline metrics measure the relevance, *i.e.,* how likely each result meets the users' information needs.

In this notebook, we go through the most common offline metrics. These can be divided again into order-unaware (e.g., Recall@k and Precision@k) and order-aware metrics (e.g., MRR, MAP, and NDCG@k).

<center><div> <img src="https://raw.githubusercontent.com/pinecone-io/examples/master/learn/algos-and-libraries/offline-evaluation/assets/metrics_diagram.png" alt="Drawing" style="width:400px;"/></div> </center>

To better understand how evaluation metrics work, let's consider an example.

Suppose that we have a small eight-image dataset (in reality this number is more likely to be in the million+ range) and searches for *"cat in the box"*. The retrieval engine returns all $k = 8$ results. These can be represented by the following images:

<center><div> <img src="https://raw.githubusercontent.com/pinecone-io/examples/master/learn/algos-and-libraries/offline-evaluation/assets/example.png" alt="Drawing" style="width:550px;"/></div> </center>

From the above, $4$ out of $8$ results are relevant, we call them *actual relevant results* as they show a cat inside a box (see results $\#2$, $\#4$, $\#5$, and $\#7$). 

The other results are not relevant because they show *only* cats (see results $\#1$, $\#6$), a box only (see result $\#3$), or a *dog* inside a box (see result $\#8$). These results do not correspond to the search query.

We have highlighted in cyan these actual relevant results.

<center><div> <img src="https://raw.githubusercontent.com/pinecone-io/examples/master/learn/algos-and-libraries/offline-evaluation/assets/example_highlighted.png" alt="Drawing" style="width:550px;"/></div> </center>

It is this dataset that we will use throughout this notebook. We will learn how to use offline metrics and define them in *Python*.

### Actual and Predicted Conditions

Before starting, it is important to clarify what we mean by *true positive*, *true negative*, *false positive*, and *false negative*.

In classification tasks, these terms result from a comparison between an *actual* condition and a *predicted* condition:

* *Actual Condition*: the true conditions of every item in the population, these are composed of positives ($p$) and negatives ($n$). For example, relevant items (cats in boxes) would be *positive*, and everything else would be *negative*.

$$totalPopulation = p + n$$

* *Predicted Condition*: the retrieval algorithm predicts whether something is *positive* ($\hat{p}$) or *negative* ($\hat{n}$). When we return $8$ candidates for the *cat in box* image, our retrieval algorithm has returned $8$ *predicted positives*, and everything else is a *predicted negative*.

If what the system predicts corresponds to the actual condition we have a *true positive* ($p\hat{p}$, we retrieve a picture of a *cat in a box*), or a *true negative* ($n\hat{n}$, we *do not* retrieve a picture that is *not* a cat in a box).

Otherwise, if the predicted does not correspond to the actual condition, we have a *false positive* ($n\hat{p}$, we retrieve a *dog* in a box) or a *false negative* ($p\hat{n}$, we *do not* retrieve a cat in a box).

The below *confusion matrix* summarises those terms.

<center><div> <img src="https://raw.githubusercontent.com/pinecone-io/examples/master/learn/algos-and-libraries/offline-evaluation/assets/confusion_matrix.png" alt="Drawing" style="width: 400px;"/></div> </center>

Let's clarify this by looking at our example of cat in the box. Suppose that the software predicts as *positive* the first $2$ results. In this case, we will have $1$ true positive, $3$ true negative, $1$ false positive, and $3$ false negative.

<center><div> <img src="https://raw.githubusercontent.com/pinecone-io/examples/master/learn/algos-and-libraries/offline-evaluation/assets/example_condition.png" alt="Drawing" style="width: 500px;"/></div> </center>

## Offline Metrics: order-unaware

### Recall@k

$Recall@K$ is the most common offline metric and shows how many actual relevant results (*true positives*) were returned out of all actual relevant results (*true positives* and *false negatives*).

Note that the *K* represents the number of results to return.

Below is the generalised formula for Recall@K: 
$$Recall@K = \frac{truePositive@K}{truePositive@K + falseNegative@K}$$

Let's go back to our example with $N = 8$, $K = 1...N$, and $4$ actual relevant results - see them below in cyan.
 
<center><div> <img src="https://raw.githubusercontent.com/pinecone-io/examples/master/learn/algos-and-libraries/offline-evaluation/assets/example_highlighted.png" alt="Drawing" style="width:550px;"/></div> </center>

When $k = 2$, the *true positive* is $1$ (when $k = 2$) while the *false negative* are $3$ (when $k = 4$, $k = 5$, and $k = 7$).

Therefore, $Recall@2$ will be calculated as:

$$Recall@2 = \frac{1}{1 + 3} = \frac{1}{4} = 0.25$$

<center><div> <img src="https://raw.githubusercontent.com/pinecone-io/examples/master/learn/algos-and-libraries/offline-evaluation/assets/recall-at-two.png" alt="Drawing" style="width: 450px;"/></div> </center>

With the same logic, we can calculate $Recall@K$ for each $K = 1...N$:

* $K = 1$, $Recall@1 = 0 / (0 + 4) = 0.00$
* $K = 2$, $Recall@2 = 1 / (1 + 3) = 0.25$
* $K = 3$, $Recall@3 = 1 / (1 + 3) = 0.25$
* $K = 4$, $Recall@4 = 2 / (2 + 2) = 0.50$
* $K = 5$, $Recall@5 = 3 / (3 + 1) = 0.75$
* $K = 6$, $Recall@6 = 3 / (3 + 1) = 0.75$
* $K = 7$, $Recall@7 = 4 / (4 + 0) = 1.00$
* $K = 8$, $Recall@8 = 4 / (4 + 0) = 1.00$

Let's calculate the same using *Python*.
We can generate a function called ```recall``` using the above example. It takes as inputs the *actual values*, the *predicted values*, and *K*.

In [1]:
# recall@K function
def recall(actual, predicted, K):
    act_set = set(actual)
    pred_set = set(predicted[:K])
    result = round(len(act_set & pred_set) / float(len(act_set)), 2)
    return result

After inizializing the variables `actual` and `predicted`, and we can loop through `K` from 1 to 8 to generate results.

In [2]:
actual = ["2", "4", "5", "7"]
predicted = ["1", "2", "3", "4", "5", "6", "7", "8"]
for K in range(1, 9):
    print(f"Recall@{K} = {recall(actual, predicted, K)}")

Recall@1 = 0.0
Recall@2 = 0.25
Recall@3 = 0.25
Recall@4 = 0.5
Recall@5 = 0.75
Recall@6 = 0.75
Recall@7 = 1.0
Recall@8 = 1.0


The outputs correspond to those manually calculated! 

As you can note, Recall@K is pretty straightforward. However, it does not consider the order of the search results. 

Order-aware metrics, such as *MRR*, *MAP@K*, and *NDCG@K*, can solve this.

## Offline Metrics: order-aware

### Mean Reciprocal Rank (MRR)

The *Mean Reciprocal Rank (MRR)* is an order-aware metric, helpful when we want our system to return the most relevant result and want that result to be at the top position. Differently from the previous metrics, the MRR is calculated on multiple queries.

The general formula for *MRR*:

$$MRR = \frac{1}{Q} \sum_{q = 1}^{Q} \frac{1}{rank_q}$$

where $Q$ is the total number of queries, $q$ is a specific query, and $rank_q$ is the rank of the first relevant result for query $q$.

Let's consider our previous example, where the user searches for *"cat in a box"*. Assume that the user adds other $2$ queries so that $Q = 3$.

The other queries can be *"white cat"* and *"dark cat"*. The search results are shown below, where the cyan highlighted images correspond to the actual relevant results based on the query.

<center><div> <img src="https://raw.githubusercontent.com/pinecone-io/examples/master/learn/algos-and-libraries/offline-evaluation/assets/mrr.png" alt="Drawing" style="width: 900px;"/></div> </center>

To understand MRR, we will build the formula step-by-step. We start by computing the reciprocal of the rank of the first actual relevant result.

$$\frac{1}{rank_q}$$

The first rank for query $\#1$ ($q=1$) corresponds to $rank_1 = 2$, the first image showing a cat in the box. The first rank for query $\#2$ ($q=2$) corresponds to $rank_2 = 1$, the first image showing a white cat. The first rank for query $\#3$ ($q=3$) corresponds to $rank_3 = 5$, the first image showing a dark cat.

Their reciprocal rank will be as follows:
* Query $\#1 \rightarrow \frac{1}{rank_1} = \frac{1}{2} = 0.5$


* Query $\#2 \rightarrow \frac{1}{rank_2} = \frac{1}{1} = 1$
* Query $\#3 \rightarrow \frac{1}{rank_3} = \frac{1}{5} = 0.2$

As a final step, we can aggregate the above-calculated reciprocal ranks to derive the *MRR*. This is a simple average calculation:

$$MRR = \frac{1}{3} (0.5 + 1 + 0.2) ≅ 0.57$$

Let's now derive a code to calculate the same using *Python*.

In [1]:
# relevant results for query #1, #2, and #3
actual_relevant = [
    [2, 4, 5, 7],
    [1, 4, 5, 7],
    [5, 8]
]

In [2]:
# number of queries
Q = len(actual_relevant)

# calculate the reciprocal of the first actual relevant rank
reciprocal = 0
for q in range(Q):
    first_result = actual_relevant[q][0]
    reciprocal = reciprocal + (1 / first_result)
    print(f"query #{q+1} = 1/{first_result} = {reciprocal}")

# calculate mrr
mrr = 1/Q * reciprocal

# generate results
print("MRR =", round(mrr,2))

query #1 = 1/2 = 0.5
query #2 = 1/1 = 1.5
query #3 = 1/5 = 1.7
MRR = 0.57


Note that the *MRR* metric does not care about the position of the remaining relevant results. This means that this metric is not helpful when dealing with use-cases requiring multiple relevant results.

### Mean Average Precision at k (MAP)

MAP is another very popular order-aware metric. But before we can understand MAP, we must understand the metrics that contribute to the MAP metric. The first is an order-*unaware* metric called Precision@k.

*Precision@K* quantifies how many items in the top-$K$ results are relevant. As with *Recall@K*, it fails to consider the order of the search results. 

The general formula for *Precision@K*:

$$Precision@K = \frac{truePositive@K}{truePositive@K + falsePositive@K}$$

Let's consider our example with one query (*"cat in the box"*) and $K = 2$. The *true positive* is $1$ (when $K = 2$) and the *false positive* is $1$ (when $K = 1$). 

Note that the denominator $(truePositive@K + falsePositive@K)$ is the total number of results returned, and so is equal to $K$.

Therefore, $Precision@2$ is calculated as:

$$Precision@2 = \frac{1}{1 + 1} = \frac{1}{2} = 0.50$$

<center><div> <img src="https://raw.githubusercontent.com/pinecone-io/examples/master/learn/algos-and-libraries/offline-evaluation/assets/precision-at-two.png" alt="Drawing" style="width: 450px;"/></div> </center>

Using the same formula, we calculate $Precision@K$ for each $K = 1...N$:

* $K = 1$, $Precision@1 = 0 / (1) = 0.00$
* $K = 2$, $Precision@2 = 1 / (1 + 1) = 0.50$
* $K = 3$, $Precision@3 = 1 / (1 + 2) = 0.33$
* $K = 4$, $Precision@4 = 2 / (2 + 2) = 0.50$
* $K = 5$, $Precision@5 = 3 / (3 + 2) = 0.60$
* $K = 6$, $Precision@6 = 3 / (3 + 3) = 0.50$
* $K = 7$, $Precision@7 = 4 / (4 + 3) = 0.57$
* $K = 8$, $Precision@8 = 4 / (4 + 4) = 0.50$

We are now ready to go through the *Mean Average Precision@K (mAP@K)*. As per *MRR*, this is an order-aware metric, calculated on multiple queries.

The general formula is the below,

$$mAP@K = \frac{1}{Q} \sum_{q = 1}^{Q} AP@K_q$$

where $Q$ is the number of queries, and $AP@K_q$ is the average precision@k across $k = [1, ... K]$ for the query $q$ and is calculated using the *Precision@k* metric discussed above (note that we are using lowercase $k$ as we are calculating for several $k$ values up to the maximum $K$ value).

$$AP@K_q = \frac{\sum_{k = 1}^{K} (Precision@k * rel_k)}{\# \space of \space relevant \space results}$$

$rel_k$ is an *indicator* or *characteristic function*, equal to $1$ when the $k^{th}$ result is an actual relevant value, and $0$ otherwise.

Let's calculate the $MAP@K$ using our example, with $3$ queries (*"cat in the box"*, *"white cat"*, and *"dark cat"*)

We can first calculate the $Precision@k$ and the $AP@K$ for each query, then calculate their average to derive the final $MAP$.

Following what previously explained, $Precision@k$ and $rel_k$ will be the following:

<center><div> <img src="https://raw.githubusercontent.com/pinecone-io/examples/master/learn/algos-and-libraries/offline-evaluation/assets/example_MAP.png" alt="Drawing" style="width: 450px;"/></div> </center>

Given that $Precision@k$ is multiplied by $rel_k$, the formula can be simplified when $rel_k = 0$ as $Precision@k * rel_k = 0$.

<center><div> <img src="https://raw.githubusercontent.com/pinecone-io/examples/master/learn/algos-and-libraries/offline-evaluation/assets/example_MAP2.png" alt="Drawing" style="width: 250px;"/></div> </center>

We then need to calculate $\sum_{k = 1}^{n} (Precision@k * rel_k)$ for each query, and divide it by the *# of relevant results* to get the $AP@K_q$ score. The # of relevant results will be $4$ for query $\#1$, $4$ for query $\#2$, and $2$ for query $\#3$.

$$AP@8_1 = \frac{0.5 * 1 + 0.5 * 1 + 0.6 * 1 + 0.57 * 1}{4} = \frac{2.17}{4} = 0.54$$

$$AP@8_2 = \frac{1 * 1 + 0.5 * 1 + 0.6 * 1 + 0.57 * 1}{4} = \frac{2.67}{4} = 0.67$$

$$AP@8_3 = \frac{0.2 * 1 + 0.25 * 1}{2} = \frac{0.45}{2} = 0.22$$

Finally, we calculate the **M**ean of the three **AP** scores:

$$MAP@8 = \frac{AP@8_1 + AP@8_2 + AP@8_3}{Q} = \frac{0.54 + 0.67 + 0.22}{3} = \frac{1.4}{3} = 0.48$$

Let's now generate the code to calculate *MAP* using Python. 

In [1]:
# initialize variables
actual = [
    [2, 4, 5, 7],
    [1, 4, 5, 7],
    [5, 8]
]
Q = len(actual)
predicted = [1, 2, 3, 4, 5, 6, 7, 8]
K = 8
ap = []

# loop through and calculate AP for each query q
for q in range(Q):
    ap_num = 0
    # loop through k values
    for k in range(K):
        # calculate precision@k
        act_set = set(actual[q])                                                                                                                                   
        pred_set = set(predicted[:k+1])
        precision_at_k = len(act_set & pred_set) / (k+1)
        # calculate rel_k values
        if predicted[k] in actual[q]:
            rel_k = 1
        else:
            rel_k = 0
        # calculate numerator value for ap
        ap_num += precision_at_k * rel_k
    # now we calculate the AP value as the average of AP
    # numerator values
    ap_q = ap_num / len(actual[q])
    print(f"AP@{K}_{q+1} = {round(ap_q,2)}")
    ap.append(ap_q)

# now we take the mean of all ap values to get mAP
map_at_K = sum(ap) / Q

# generate results
print(f"mAP@{k} = {round(map_at_K, 2)}")

AP@8_1 = 0.54
AP@8_2 = 0.67
AP@8_3 = 0.23
mAP@8 = 0.48


### Normalized Discounted Cumulative Gain (NDCG@K)

The $NDCG@K$ is an order-aware metric derived from the $DCG@K$, which derives from $CG@K$. 

That is why we will first explain $CG@K$, then $DCG@K$, and finally $NDCG@K$.

The $CG@K$ metric is calculated as the sum of the top-$K$ relevance scores. Differently from before, the results from the user's search will not be divided into relevant and not relevant but will be rated from the less to the most relevant. We can use different colors to characterize the different scores: dark grey will be the color associated to the less relevant result $(0)$, cyan will be the color associated to the most relevant result $(4)$.

<center><div> <img src="https://raw.githubusercontent.com/pinecone-io/examples/master/learn/algos-and-libraries/offline-evaluation/assets/relevance_score.png" alt="Drawing" style="width: 400px;"/></div> </center>

Let's now consider another example with $1$ query (this time, the user will search for *"***white*** cat in the box"*), $K = 8$, $k = 1...K$, and let's include the relevance score. Based on judgement, and considering their position, $k$, the relevance score can be assigned as follows:

<center><div> <img src="https://raw.githubusercontent.com/pinecone-io/examples/master/learn/algos-and-libraries/offline-evaluation/assets/ndcg_relevance.png" alt="Drawing" style="width: 450px;"/></div> </center>

We can then calculate the cumulative gain *up to* position $K$, or $CG@K$, by summing up the relevance scores up to $K$.

$$CG@k = \sum_{k = 1}^{K} rel_k$$

When $K = 2$, the $CG@2$ is:

$$CG@2 = relevance_{k = 1} + relevance_{k = 2} = 0 + 4 = 4$$

With the same logic, we can calculate $CG@K$ for each $k = 1...K$:

* $K = 1$, $CG@1 = 0$
* $K = 2$, $CG@2 = 0 + 4 = 4$
* $K = 3$, $CG@3 = 0 + 4 + 1 = 5$
* $K = 4$, $CG@4 = 0 + 4 + 1 + 3 = 8$
* $K = 5$, $CG@5 = 0 + 4 + 1 + 3 + 4 = 12$
* $K = 6$, $CG@6 = 0 + 4 + 1 + 3 + 4 + 1 = 13$
* $K = 7$, $CG@7 = 0 + 4 + 1 + 3 + 4 + 1 + 3 = 16$
* $K = 8$, $CG@8 = 0 + 4 + 1 + 3 + 4 + 1 + 3 + 2 = 18$

It's worth noting how this metric does not consider the position of the results. For example, if we swap the first two results so that the relevance for $k = 1$ equals $4$ and the relevance for $k = 2$ equals $0$, we can see that $CG@2$ is still $4$, even though having relevance $4$ as first result is often better than having it as the second.

<center><div> <img src="https://raw.githubusercontent.com/pinecone-io/examples/master/learn/algos-and-libraries/offline-evaluation/assets/ndcg_relevance_two.png" alt="Drawing" style="width: 450px;"/></div> </center>

To account for this, the $DCG@K$ metric can be used. It adds a penalty (this being $log_2(k + 1)$) on the score depending on the position $k$. The higher the position, the higher the penalty. This was explained by [*Wang et al. (2013)*](https://arxiv.org/abs/1304.6480).

$$DCG@K = \sum_{k = 1}^{K} \frac{rel_k}{log_2(1 + k)}$$

Note that $log_2(2) = 1$, so when $k = 1$ the penalty is not applied.

When $K = 2$, then $DCG@2$ is calculated as follows:

$$DCG@2 = \frac{0}{log_2(1+1)} + \frac{4}{log_2(1+2)} = \frac{4}{1.58} = 2.52$$

We can replicate this in Python:

In [2]:
from math import log2

# initialize variables
relevance = [0, 4, 1, 3, 4, 1, 3, 2]
K = 8

dcg = 0
# loop through each item and calculate DCG
for k in range(1, K+1):
    rel_k = relevance[k-1]
    # calculate DCG
    dcg += rel_k / log2(1 + k)
    print(f"DCG@{k} = {round(dcg, 2)}")

DCG@1 = 0.0
DCG@2 = 2.52
DCG@3 = 3.02
DCG@4 = 4.32
DCG@5 = 5.86
DCG@6 = 6.22
DCG@7 = 7.22
DCG@8 = 7.85


An alternative formulation of the DCG@K, commonly used in industry, is the below:

$$DCG@k = \sum_{k = 1}^{K} \frac{2^{rel_k} - 1}{log_2(1 + k)}$$

When the relevance values of documents are binary, *i.e.,* $rel_k ∈ (0,1)$, both versions of the DCG@K formulae give the same result.

The $NDCG@K$ metric has been introduced to allow comparability between queries' results. In fact, the scores obtained using $DCG@K$ can vary from $0$ to $\infty$ and can be difficult, if not impossible, to establish which and why a score is better than another.

As the name suggests, $NDCG@K$ normalizes the scores resulting from $DCG@K$ using an *ideal* order of the relevant results ($IDCG@K$). 

$$NDCG@K = \frac{DCG@K}{IDCG@K}$$

If we consider our example, ideally, we would have wanted our results to be sorted by relevance, as follows:

<center><div> <img src="https://raw.githubusercontent.com/pinecone-io/examples/master/learn/algos-and-libraries/offline-evaluation/assets/ndcg_relevance_sorted.png" alt="Drawing" style="width: 500px;"/></div> </center>

If we recalculate the $DCG@K$ using the new order, we will get the *ideal* $DCG@K$, or $IDCG@K$.

In a perfect ranking algorithm, $DCG@K = IDCG@K$, therefore $NDCG@K = 1$.

When $K = 2$,

$$IDCG@2 = \frac{4}{log_2(1+1)} + \frac{4}{log_2(1+2)} = 4 + 2.52 = 6.52$$

In Python we calculate this using:

In [3]:
# sort items in 'relevance' from most relevant to less relevant
ideal_relevance = sorted(relevance, reverse=True)

print(ideal_relevance)

idcg = 0
# as before, loop through each item and calculate *Ideal* DCG
for k in range(1, K+1):
    rel_k = ideal_relevance[k-1]
    # calculate DCG
    idcg += rel_k / log2(1 + k)
    print(f"IDCG@{k} = {round(idcg, 2)}")

[4, 4, 3, 3, 2, 1, 1, 0]
IDCG@1 = 4.0
IDCG@2 = 6.52
IDCG@3 = 8.02
IDCG@4 = 9.32
IDCG@5 = 10.09
IDCG@6 = 10.45
IDCG@7 = 10.78
IDCG@8 = 10.78


Now, we can derive the $NDCG@2$,

$$NDCG@2 = \frac{DCG@2}{IDCG@2} = \frac{2.52}{6.52} = 0.39$$

As before, we can calculate $NDCG@K$ for $k = 1,...,K$ using Python:

In [4]:
dcg = 0
idcg = 0

for k in range(1, K+1):
    # calculate rel_k values
    rel_k = relevance[k-1]
    ideal_rel_k = ideal_relevance[k-1]
    # calculate dcg and idcg
    dcg += rel_k / log2(1 + k)
    idcg += ideal_rel_k / log2(1 + k)
    # calcualte ndcg
    ndcg = dcg / idcg
    print(f"NDCG@{k} = {round(ndcg, 2)}")

NDCG@1 = 0.0
NDCG@2 = 0.39
NDCG@3 = 0.38
NDCG@4 = 0.46
NDCG@5 = 0.58
NDCG@6 = 0.6
NDCG@7 = 0.67
NDCG@8 = 0.73


---